In [2]:
#Wandb api key = 97b0aeca9808bb2988f3f438db215b777df7b0e9
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Define the dataset
# Load a dataset for fine-tuning; here, we use a dataset from the Hugging Face library.
# Load the train and validation splits of the dataset
train_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
eval_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="validation")

# Add the padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token # Using eos_token as pad_token
# or
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Adding a new pad token '[PAD]'

# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Rename the 'input_ids' column to 'labels' but keep the original 'input_ids'
tokenized_train_dataset = tokenized_train_dataset.rename_column("input_ids", "labels")
tokenized_eval_dataset = tokenized_eval_dataset.rename_column("input_ids", "labels")

# Create a new 'input_ids' column by duplicating the 'labels' column
tokenized_train_dataset = tokenized_train_dataset.add_column("input_ids", tokenized_train_dataset["labels"])
tokenized_eval_dataset = tokenized_eval_dataset.add_column("input_ids", tokenized_eval_dataset["labels"])

# Remove the original 'text' column
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text"])
tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(["text"])

# Define training arguments
training_args = TrainingArguments(
    output_dir="./content_gen_model",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
)

# Define Trainer for optimized training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset, # Pass the training dataset
    eval_dataset=tokenized_eval_dataset, # Pass the evaluation dataset
    tokenizer=tokenizer,
)

# Train the model
trainer.train() # took 2 hours

# Save the fine-tuned model and tokenizer
model.save_pretrained("./content_gen_model")
tokenizer.save_pretrained("./content_gen_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.146000,1.294499
2,1.002900,1.295373
3,1.111100,1.302951


('./content_gen_model/tokenizer_config.json',
 './content_gen_model/special_tokens_map.json',
 './content_gen_model/vocab.json',
 './content_gen_model/merges.txt',
 './content_gen_model/added_tokens.json')

In [4]:
# Load the fine-tuned model for content generation
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./content_gen_model")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./content_gen_model")

# Define content generation function with sampling controls
def generate_content(prompt, max_length=50, temperature=0.7, top_k=50):
    inputs = fine_tuned_tokenizer(prompt, return_tensors="pt")
    output = fine_tuned_model.generate(
        inputs.input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True
    )
    return fine_tuned_tokenizer.decode(output[0], skip_special_tokens=True)

# Example content generation
prompt = "The future of artificial intelligence is"
generated_text = generate_content(prompt)
print("Generated Text:", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Text: The future of artificial intelligence is uncertain. There is no firm definition of what AI should mean and to what extent it should depend on human knowledge. However, it is likely that by the end of the 20th century the world will be in a state


In [5]:
!zip -r content_gen_model.zip /content/content_gen_model

  adding: content/content_gen_model/ (stored 0%)
  adding: content/content_gen_model/config.json (deflated 52%)
  adding: content/content_gen_model/vocab.json (deflated 68%)
  adding: content/content_gen_model/checkpoint-27540/ (stored 0%)
  adding: content/content_gen_model/checkpoint-27540/trainer_state.json (deflated 83%)
  adding: content/content_gen_model/checkpoint-27540/config.json (deflated 52%)
  adding: content/content_gen_model/checkpoint-27540/optimizer.pt (deflated 8%)
  adding: content/content_gen_model/checkpoint-27540/vocab.json (deflated 68%)
  adding: content/content_gen_model/checkpoint-27540/training_args.bin (deflated 52%)
  adding: content/content_gen_model/checkpoint-27540/model.safetensors (deflated 7%)
  adding: content/content_gen_model/checkpoint-27540/special_tokens_map.json (deflated 74%)
  adding: content/content_gen_model/checkpoint-27540/tokenizer_config.json (deflated 55%)
  adding: content/content_gen_model/checkpoint-27540/merges.txt (deflated 53%)
  

In [6]:
from google.colab import files
files.download('content_gen_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>